# Knowledge Graph extraction from documents
Based on https://github.com/lamm-mit/GraphReasoning

This notebook creates a knowledge graph using functions in the GraphReasoning_Mod module which is adpated from https://github.com/lamm-mit/GraphReasoning to work with huggingface endpoints and for use with huggingface documents. 
- documents are scrapped using data.ipynb from pdfs, blogs, and youtube videos transcripts. The only metadata used in the knowledge graph creating is adding relationships between source document and node.
- LLM model Mistral-Nemo-Instruct-2407
- Embedding model: dunzhang/stella_en_1.5B_v5 (chosen based on size and position on leaderboard DEC 2024)
- KG created without refinement loops in when identifying nodes. 
- KG uses simplify graph and additonal tools in GraphReasoning_Mod

_Note:Some file paths may have to be updated due to restructing of the repo_


## Setup

In [10]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import GraphReasoning_Mod
import importlib



In [2]:
# Reload the module to reflect any changes
importlib.reload(GraphReasoning_Mod)

# Re-import all objects from the module
from GraphReasoning_Mod import *

In [3]:
from langchain_huggingface import HuggingFaceEndpoint

#Initialize the model endpoint
HOST_URL_INF = ":8080" #Mistral-NeMo-Instruct-2407
MAX_NEW_TOKENS = 1012

TEMPERATURE = 0.2
TIMEOUT = 120
TOP_P = .9

llm = HuggingFaceEndpoint(
    endpoint_url=HOST_URL_INF,
    task="text-generation",
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,
    temperature = TEMPERATURE,
    timeout=TIMEOUT,
    top_p=TOP_P
)
#print(llm.invoke("What is HuggingFace?"))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "dunzhang/stella_en_1.5B_v5" #"BAAI/bge-small-en-v1.5" #dunzhang/stella_en_1.5B_v5
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embd = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

## Import Documents

In [1]:
import pickle
import os

# load pickled documents
pickle_file_path = '../../data/storage/full_all_documents.pkl'
if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        all_pdf_docs, all_yt_docs, all_blog_docs = pickle.load(f)
else:
    print("Pickle file not found.")

#check if the documents are loaded
print("Number of PDF documents:", len(all_pdf_docs))
print("Number of YouTube documents:", len(all_yt_docs))
print("Number of blog documents:", len(all_blog_docs))

#print dictionary keys from metadata
print("Metadata keys for PDF documents:", all_pdf_docs[0].metadata.keys())
print("Metadata keys for yt documents:", all_yt_docs[0].metadata.keys())
print("Metadata keys for blog documents:", all_blog_docs[0].metadata.keys())

# Combine all documents into a single list
all_docs = all_pdf_docs+  all_yt_docs+  all_blog_docs

print(f"Total number of documents: {len(all_docs)}")


Number of PDF documents: 2048
Number of YouTube documents: 442
Number of blog documents: 11
Metadata keys for PDF documents: dict_keys(['source', 'page'])
Metadata keys for yt documents: dict_keys(['source', 'title', 'description', 'upload_date', 'duration', 'uploader', 'view_count', 'like_count', 'dislike_count', 'categories', 'tags', 'sequence'])
Metadata keys for blog documents: dict_keys(['source', 'title', 'description', 'language'])
Total number of documents: 2501


## Test Functions

In [16]:

# #TEST the function
# test_chunks = split_text_for_kg(all_pdf_docs[:2])
# display((test_chunks[0]))

In [17]:
# #TEST
# test_df = documents2Dataframe(test_chunks)
# display(test_df.head())

In [18]:
# #TEST df2Graph
# concepts_list = df2Graph(test_df, llm, repeat_refine=0, verbatim=False)
# display(concepts_list[0])

In [19]:
# dfg1 = graph2Df(concepts_list)
# display(dfg1.head())

In [20]:
# display(dfg1.head())
# test_df = add_source_relationship(dfg1)
# display(test_df)

In [21]:
#TEST make_graph_from_text
graph_GraphML, G = make_graph_from_doc_batch(all_pdf_docs[2:3],llm,
                          include_contextual_proximity=False,
                          chunk_size=2500,chunk_overlap=0,
                          repeat_refine=1,verbatim=False,
                          data_dir='./data/GR_output_KG/',
                          simple= True
                         )

In [22]:
# # Extract node-edge triplet information
# triplets = []
# for u, v, data in G.edges(data=True):
#     triplets.append({
#         'node_1': u,
#         'node_2': v,
#         'edge': data.get('title', ''),
#         'weight': data.get('weight', 1),  # Default weight to 1 if not present
#         'data': data
#     })

# # Convert to DataFrame for better visualization
# triplets_df = pd.DataFrame(triplets)
# display(triplets_df.head(5))

In [23]:
#TEST regenerate_node_embeddings

node_embeddings = regenerate_node_embeddings(G, G.nodes(), embd)
print("Number of node embeddings:", len(node_embeddings))
node_embeds2 = update_node_embeddings(node_embeddings, G, embd, remove_embeddings_for_nodes_no_longer_in_graph=True, verbatim=False)


Number of node embeddings: 27


In [24]:
#TEST add_new_subgraph_from_text 

G_new, node_embeddings, res= add_new_subgraph_from_docs(
    all_pdf_docs[8:9], llm, embd, 
    data_dir_output='./data/temp/', verbatim=False, size_threshold=10,
    chunk_size=2500, do_Louvain_on_new_graph=True,
    include_contextual_proximity=False, repeat_refine=0,
    similarity_threshold=0.95, do_simplify_graph=True,
    return_only_giant_component=False, save_common_graph=False,
    G_exisiting=None, existing_node_embeddings=None
    )

if G_new:
    print("----New subgraph created successfully.")
    print("Nodes:", G_new.nodes())
    print("Edges:", G_new.edges(data=True))
else:
    print("Failed to create a new subgraph.")

----New subgraph created successfully.
Nodes: ['C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data\\pdfs\\Gilbert_Strang_Linear_Algebra_and_Its_Applicatio_230928_225121.pdf', 'linear algebra', 'abstraction', 'column 1', 'gilbert strang', 'orthogonal matrices', 'rows 1 and 2', 'same plane']
Edges: [('C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data\\pdfs\\Gilbert_Strang_Linear_Algebra_and_Its_Applicatio_230928_225121.pdf', 'linear algebra', {'title': 'is source document of', 'weight': 1.0}), ('C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data\\pdfs\\Gilbert_Strang_Linear_Algebra_and_Its_Applicatio_230928_225121.pdf', 'abstraction', {'title': 'is source document of', 'weight': 1.0}), ('C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data\\pdfs\\Gilbert_Strang_Linear_Algebra_and_Its_Applicatio_230928_225121.pdf', 'column 1', {'title': 'is source document of', 'weight': 1.0}), ('C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data\\pdfs

## Create create and save networkx graph

In [25]:

# Initialize variables
G_existing = None
existing_node_embeddings = None
failed_batches = []  


# Process documents in chunks
chunk_size = 1500
batch_size = 10

# Split all_docs into batches of size batch_size
doc_batches = [all_docs[i:i + batch_size] for i in range(0, len(all_docs), batch_size)]

for batch_idx, doc_batch in tqdm(enumerate(doc_batches), total=len(doc_batches)):
    try:
        G_existing, existing_node_embeddings, res = add_new_subgraph_from_docs(
            input_docs=doc_batch,
            llm=llm,
            embd=embd,
            data_dir_output='./data/GR_no_refine/',
            verbatim=False,
            size_threshold=10,
            chunk_size=chunk_size,
            do_Louvain_on_new_graph=True,
            include_contextual_proximity=False,
            repeat_refine=0,
            similarity_threshold=0.95,
            do_simplify_graph=True,
            return_only_giant_component=False,
            save_common_graph=False,
            G_exisiting=G_existing,
            graph_GraphML_exisiting=None,
            existing_node_embeddings=existing_node_embeddings
        )

        print(f"Processed batch {batch_idx}, updated graph stats:", res)
        #save embeddings and failed batches
        with open('./data/GR_no_refine/embeddings.pkl', 'wb') as f:
            pickle.dump(existing_node_embeddings, f)
        with open('./data/GR_no_refine/failed_batches.pkl', 'wb') as f:
            pickle.dump(failed_batches, f)
    except Exception as e:
        # Log the failed batch index
        failed_batches.append(batch_idx)
        print(f"Error processing batch {batch_idx} with batch size {batch_size}: {e}")

# Final graph statistics and saving
print("Final graph statistics:", res if 'res' in locals() else "No successful batches")
print("Failed batch indices:", failed_batches)

  0%|          | 0/251 [00:00<?, ?it/s]

Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 


  0%|          | 1/251 [03:31<14:42:16, 211.74s/it]

Processed batch 0, updated graph stats: ({'Number of Nodes': 26, 'Number of Edges': 60, 'Average Degree': 4.615384615384615, 'Density': 0.18461538461538463, 'Connected Components': 1, 'Number of Communities': 4}, False)


  1%|          | 2/251 [08:10<17:21:29, 250.96s/it]

...

Processed batch 246, updated graph stats: ({'Number of Nodes': 3324, 'Number of Edges': 9288, 'Average Degree': 5.588447653429603, 'Density': 0.0016817477139421013, 'Connected Components': 1, 'Number of Communities': 23}, False)
Error parsing JSON response: 
Error parsing JSON response: [
        {
            "node_1": "matrix vector product",
            "node_2": "linear transformation",
            "edge": "is"
        },
        {
            "node_1": "change of basis",
            "node_2": "linear transformation",
            "edge": "is really just"
        },
        {
            "node_1": "orthogonal matrix",
            "node_2": "vectors",
            "edge": "multiplies"
        },
        {
            "node_1": "vectors",
            "node_2": "lengths",
            "edge": "preserves"
        },
        {
            "node_1": "vectors",
            "node_2": "angles",
            "edge": "preserves"
        },
        {
            "node_1": "orthogonal matrix",
    

 99%|█████████▉| 248/251 [28:20:41<58:06, 1162.25s/it]  

Processed batch 247, updated graph stats: ({'Number of Nodes': 3343, 'Number of Edges': 9345, 'Average Degree': 5.590786718516303, 'Density': 0.00167288651062726, 'Connected Components': 1, 'Number of Communities': 25}, False)
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: [
        {
            "node_1": "A concept from extracted ontology",
            "node_2": "A related concept from extracted ontology",
            "edge": "Relationship between the two concepts, node_1 and node_2, succinctly described"
        },
        {...}
    ]
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 
Error parsing JSON response: 


 99%|█████████▉| 248/251 [28:28:34<20:40, 413.37s/it] 


KeyboardInterrupt: 

In [10]:
#processed batch 247
#NOTE: dont have failed batches from 0-123 in the failed_batches.pkl file


# Initialize variables
G_existing = None
g_graphml_existing = "./data/GR_no_refine/final_augmented_graph.graphml"
failed_batches_path = "./data/GR_no_refine/failed_batches.pkl"
node_embd_path = "./data/GR_no_refine/embeddings.pkl"
failed_batches = []
existing_node_embeddings = {}

if os.path.exists(failed_batches_path):
    with open(failed_batches_path, 'rb') as f:
        failed_batches = pickle.load(f)

if os.path.exists(node_embd_path):
    with open(node_embd_path, 'rb') as f:
        existing_node_embeddings = pickle.load(f)
  


# Process documents in chunks
chunk_size = 1500
batch_size = 10

# Split all_docs into batches of size batch_size
doc_batches = [all_docs[i:i + batch_size] for i in range(0, len(all_docs), batch_size)]

#load existing graph and embeddings
cur_batch = 247

G_existing = nx.read_graphml(g_graphml_existing)
print(f"Existing graph loaded with {len(G_existing.nodes())} nodes and {len(G_existing.edges())} edges.")
print(f"Num failed_batches: {len(failed_batches)}")
print(f"Num existing_node_embeddings: {len(existing_node_embeddings)}")


Existing graph loaded with 3343 nodes and 9345 edges.
Num failed_batches: 0
Num existing_node_embeddings: 3343


In [11]:
start_batch = cur_batch + 1
for batch_idx, doc_batch in tqdm(enumerate(doc_batches[start_batch:]), total=len(doc_batches[start_batch:])):
    try:
        G_existing, existing_node_embeddings, res = add_new_subgraph_from_docs(
            input_docs=doc_batch,
            llm=llm,
            embd=embd,
            data_dir_output='./data/GR_no_refine/',
            verbatim=False,
            size_threshold=10,
            chunk_size=chunk_size,
            do_Louvain_on_new_graph=True,
            include_contextual_proximity=False,
            repeat_refine=0,
            similarity_threshold=0.95,
            do_simplify_graph=True,
            return_only_giant_component=False,
            save_common_graph=False,
            G_exisiting=G_existing,
            graph_GraphML_exisiting=None,
            existing_node_embeddings=existing_node_embeddings
        )

        print(f"Processed batch {batch_idx}, updated graph stats:", res)
        #save embeddings and failed batches
        with open('./data/GR_no_refine/embeddings.pkl', 'wb') as f:
            pickle.dump(existing_node_embeddings, f)
        with open('./data/GR_no_refine/failed_batches.pkl', 'wb') as f:
            pickle.dump(failed_batches, f)
            
    except Exception as e:
        # Log the failed batch index
        failed_batches.append(batch_idx)
        print(f"Error processing batch {batch_idx} with batch size {batch_size}: {e}")

# Final graph statistics and saving
print("Final graph statistics:", res if 'res' in locals() else "No successful batches")
print("Failed batch indices:", failed_batches)

 33%|███▎      | 1/3 [25:56<51:52, 1556.43s/it]

Processed batch 0, updated graph stats: ({'Number of Nodes': 3374, 'Number of Edges': 9422, 'Average Degree': 5.5850622406639, 'Density': 0.001655814479888497, 'Connected Components': 1, 'Number of Communities': 19}, False)


 67%|██████▋   | 2/3 [1:14:05<39:00, 2340.15s/it]

Processed batch 1, updated graph stats: ({'Number of Nodes': 3483, 'Number of Edges': 9819, 'Average Degree': 5.638242894056847, 'Density': 0.0016192541338474577, 'Connected Components': 1, 'Number of Communities': 22}, False)


100%|██████████| 3/3 [1:18:25<00:00, 1568.66s/it]

Processed batch 2, updated graph stats: ({'Number of Nodes': 3484, 'Number of Edges': 9841, 'Average Degree': 5.649253731343284, 'Density': 0.0016219505401502392, 'Connected Components': 1, 'Number of Communities': 18}, False)
Final graph statistics: ({'Number of Nodes': 3484, 'Number of Edges': 9841, 'Average Degree': 5.649253731343284, 'Density': 0.0016219505401502392, 'Connected Components': 1, 'Number of Communities': 18}, False)
Failed batch indices: []
